## Summary of fitting experiments

### Tested:

- astropy.modeling.fitting
  - Levenberg-Marquardt
  - Simplex
  - SLSQP
- scipy.optimize.curve_fit
- lmfit
- ODRPACK
- saba (Sherpa)
  - Levenberg-Marquardt
  - Moncar
  - Nelder-Mead
  
Saba requires python 2.7, thus can't be used by specviz.  

### Fitter behavior:

From tests in https://github.com/ibusko/experiments_specviz/blob/master/experiment_fiting.ipynb and https://github.com/ibusko/experiments_specviz/blob/master/curve_fit.ipynb


|Fitter            | Initialization |    ROI 1             |     ROI 2           | 
|:-----------------|:---------------|:---------------------|:--------------------|
| astropy L-M      | independent    | FIT WORKS            | FIT WORKS           |
| astropy L-M      | compound       | FIT WORKS            | FIT WORKS           |
| astropy L-M      | default        | moves closer         | moves closer        |
|                  |                |                      |                     |
| astropy Simplex  | independent    | FIT WORKS            | FIT WORKS           |
| astropy Simplex  | compound       | FIT WORKS            | FIT WORKS           |
| astropy Simplex  | default        | Gaussian disappears  | Gaussian disappears |
|                  |                |                      |                     |
| astropy SLSQP    | independent    | does nothing         | does nothing        |
| astropy SLSQP    | compound       | does nothing         | does nothing        |
| astropy SLSQP    | default        | diverges             | diverges            |
|                  |                |                      |                     |
| scipy curve_fit  | default        | FIT WORKS            | FIT WORKS           |
|                  |                |                      |                     |
| lmfit            | independent    | FIT WORKS            | FIT WORKS           |
| lmfit            | compound       | FIT WORKS            | doesn't work        |
|                  |                |                      |                     |
| odr              | compound       | FIT WORKS            | doesn't work        |
|                  |                |                      |                     |
| Sherpa L-M       | independent    | FIT WORKS            | FIT WORKS           |
| Sherpa L-M       | compound       | FIT WORKS            | FIT WORKS           |
| Sherpa L-M       | default        | moves closer         | moves closer        |
|                  |                |                      |                     |
| Sherpa Moncar    | independent    | FIT WORKS            | moves closer        |
| Sherpa Moncar    | compound       | FIT WORKS            | moves closer        |
| Sherpa Moncar    | default        | Gaussian disappears  | Gaussian disappears |
|                  |                |                      |                     |
| Sherpa N-M       | independent    | moves closer         | FIT WORKS           |
| Sherpa N-M       | compound       | moves closer         | FIT WORKS           |
| Sherpa N-M       | default        | diverges             | diverges            |

### Some features of these fitters:

| Fitter             | method       | calls                  | depends on    | fix | ties | bounds | 
|:-------------------|:-------------|:-----------------------|:--------------|:---:|:----:|:------:|
| astropy L-M        |              | optimize.leastsq       | MINPACK lmdif |   X |  X   |        |
| optimize.curve_fit | lm (default) | optimize.leastsq       | MINPACK lmdif |     |      |        |
| optimize.curve_fit | trf          | optimize.least_squares |               |     |      |   X    |
| optimize.curve_fit | dogbox       | optimize.least_squares |               |     |      |   X    | 
| odr                | lm (\*)      |                        |               |   X |  X   |        | 

(\*) defaults to L-M when errors in the independent variable are ignored

<h4> ODR: </h4> Orthogonal Distance Regression is designed to handle the case where errors are present in both the independent and dependent variables. This is probably not the most common use case in spectroscopy in the UV-IR band. ODRPACK uses a modified trust-region L-M optimizer. According to the original paper, it falls back into ordinary L-M (as in optimize.leastsq) when errors in wavelength are to be ignored. Like astropy's fitters, it supports constrained (fixed) parameters, but no bounded parameters.

<h4>LMFIT:</h4> Entry point for a large number of optimizers in scipy and elsewhere. The default (as tested) reverts to L-M since it calls optimize.leastsq (minpack) via optimize.least_squares. This would be an interesting avenue to pursue since it offers so large a variety of optimizers.